### Build and train the ResNeXT 50 and 101 network and retrain it by breast cancer data base

In [ ]:
# Run the models on the Colab
from google.colab import drive
drive.mount('/content/drive')

### Import the liberaries

In [ ]:
# import pretrainedmodels
# !pip install pretrainedmodels

import torch
from torch import nn, optim
from torchvision import datasets ,transforms, models
import numpy as np
import matplotlib.pyplot as plt
import helper
import math
import time as evaltime
from torch import nn
from collections import OrderedDict

import tqdm
from tqdm import tqdm
import pandas as pd

#################################
# If GPU is available, run the model in there 
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Load the pre trained model and change the FC layer to taylor the number of classes: 8

In [ ]:
# Load the pretrained model 
ResNext101_2c= models.resnext101_32x8d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/checkpoints/resnext101_32x8d-8ba56ff5.pth


In [ ]:
# Define the classifier
classifier= nn.Sequential(OrderedDict([
    ('0', nn.Linear(2048,1024)),
    ('1',nn.ReLU(inplace=True)),
    ('2', nn.Dropout(p=0.5)),
    ('3', nn.Linear(1024,1024)),
    ('4',nn.ReLU(inplace=True)),
    ('5', nn.Dropout(p=0.5)),
    ('6', nn.Linear(1024,8)),
    ('output', nn.LogSoftmax(dim=1))
    
]))

# Add the classifier to the model as the final layers 
ResNext101_2c.fc=classifier

# Only the parameteres in the classifier needs to be retrained. 
for params in ResNext101_2c.parameters():
    params.required_grad=False
for params in ResNext101_2c.fc.parameters():
    params.required_grad=True

## Define the loss and optimization. in order to retrain the model, define the require grad true for all the parameters in the network.

In [ ]:
critirion= nn.NLLLoss()
optimizer = optim.Adam(ResNext101_2c.parameters(), lr=0.0002)
ResNext101_2c.to(device)

# Define the number of the classes and the model as the parameters, So they can be changed easily 
checkpoint=False
classes=8
model='resnext32'
str_checkpoin=''
check_dir=''
start_epoch=1

# Defne the model and classes that we defined as the parameters
if checkpoint:
  if classes==8:
    str_checkpoin='To/checkpoints_8classes/'
  else:
    str_checkpoin='To/checkpoints/'

  if model=='resnext32':
    check_dir= str_checkpoin+'checkpoint_resnext101_32x8d_2c.pth'
  elif model=='resnext64':
    check_dir= str_checkpoin+'checkpoint_resnext101_64x4d_2c.pth'
  elif model=='Inception_res_v2':
    check_dir= str_checkpoin+'checkpoint_Inception_Res_V2_2c.pth'
  elif model=='dpn':
    check_dir= str_checkpoin+'checkpoint_dpn131_c2.pth'
  elif model=='senet':
    check_dir= str_checkpoin+'checkpoint_SeNet154_2c_finetune.pth'
  elif model=='nasnet':
    check_dir= str_checkpoin+'checkpoint_nasnetalarge_2c.pth'
  stat_dict=torch.load(check_dir)
  ResNext101_2c.load_state_dict(stat_dict)


### Load the images to the train and test loader, with the ratio of 65/35

In [ ]:
dir_eight_Classes='To the brest cancer images for eight classifications/'
dir_checkpoints='To the /checkpoints/'
dir_statistics='To the/statistics/'

# split the train and test datasets with the ration of 90/10
ratio=0.9 
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomRotation(45),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomResizedCrop((224,224),scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])


# load the images to the train and test loader 
dataset= datasets.ImageFolder(dir_two_Classes,transform=train_transform)
train_sample_len=math.ceil(len(dataset)*ratio)
test_sample_len=len(dataset)-train_sample_len
trainset,testset=torch.utils.data.random_split(dataset,[train_sample_len,test_sample_len] )

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, num_workers=4, shuffle=True)
testloader =torch.utils.data.DataLoader(testset, batch_size=1, num_workers=4, shuffle=False)
classes=trainset.dataset.classes
print( len(dataset), classes)

7919 ['adenosis', 'ductal_carcinoma', 'fibroadenoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma', 'phyllodes_tumor', 'tubular_adenoma']


### Displaying one image out of all to be confirmed that the trainset is already loaded

In [ ]:
images=iter(testloader).next()
type(images)
images[0][0].shape
img=images[0][0].permute(1,2,0)

plt.imshow(img)

### Train and test the model

In [ ]:



epochs = 94
steps= 0
print_every=100
running_loss=0
min_val_loss = np.Inf
results={'epoch':[],'training_loss':[], 'test_loss':[], 'test_accuracy':[], 'training_accuracy':[]}

# Iterate through the epoches
for e in range(1,epochs+1):
  running_results={'loss':0, 'accuracy':0,'minutes':0,'steps':0}
  ResNext101_2c.train()
  train_bar=tqdm(trainloader)
  running_loss=0
    
   # Trian the model 
  for images, labels in train_bar:
      start_time= evaltime.time()
      images, labels = images.to(device), labels.to(device)
      running_results['steps']+=1
      optimizer.zero_grad()
      output=ResNext101_2c.forward(images)
      ps=torch.exp(output)
      loss=critirion(output,labels)
      loss.backward()
      optimizer.step()
      # running_loss+=loss.item()
      running_results['loss']+=loss.item()
      equality = (ps.max(dim=1)[1]==labels.data)
      running_results['accuracy']+=equality.type(torch.FloatTensor).mean()
      train_bar.set_description(desc='[%d/%d], training_loss: %.4f, training_accuracy: %.4f '% 
                                (e,epochs, running_results['loss']/running_results['steps'],running_results['accuracy']/running_results['steps']))
  test_bar=tqdm(testloader, desc='Validation Results:')
  ResNext101_2c.eval()
  valing_results={'test_loss':0, 'test_accuracy':0, 'min_loss':0, 'steps':0}

  # Do not change the parameters while evaluating the model 
  with torch.no_grad():
    # Test the model
    for images, labels in test_bar: 
      valing_results['steps']+=1
      images, labels = images.to(device), labels.to(device)
      output=ResNext101_2c.forward(images)
      loss=critirion(output,labels)
      valing_results['test_loss']+= loss.item()
      ps=torch.exp(output)
      equality = (ps.max(dim=1)[1]==labels.data)
      valing_results['test_accuracy']+=equality.type(torch.FloatTensor).mean()
      test_bar.set_description(desc='test_loss: %.4f, test_accuracy: %.4f'%
                               (valing_results['test_loss']/valing_results['steps'], valing_results['test_accuracy']/valing_results['steps'] ))
    if (valing_results['test_loss']/valing_results['steps']< min_val_loss):
            min_val_loss=valing_results['test_loss']/valing_results['steps']
            print('saving the model with min loss of : '+ str(min_val_loss))
            torch.save(ResNext101_2c.state_dict(),dir_checkpoints+"dataaug_FT1024_checkpoint_resnext101_32x8d_2c.pth")

  # update the results for each epoch          
  results['training_loss'].append(running_results['loss']/running_results['steps'])
  results['test_loss'].append(valing_results['test_loss']/valing_results['steps'])
  results['test_accuracy'].append(valing_results['test_accuracy'].item()/valing_results['steps'])
  results['training_accuracy'].append(running_results['accuracy'].item()/running_results['steps'])
  
  data_frame=pd.DataFrame(
        data={
            # 'Epoch':1,
            'Training_Loss':results['training_loss'],
            'Test_Loss': results['test_loss'],
            'Test_Accuracy':results['test_accuracy'],
            'Training_Accuracy':results['training_accuracy'],
           
                 },
        index=range(1,e+1)
    )
  # Save the results into a CSV file 
  data_frame.to_csv(dir_statistics+ 'dataaug_FT1024_Resnext101_32x8d_eval_results.csv',  index_label="Epoch")    
